# Data Cleaning

## Downloading

In [ ]:
import requests
import os

data_list = []
# The path containing my datas
path_data = './data'
if not os.path.exists(path_data):
    os.makedirs(path_data)
# The name of the files
link_2014 = "https://s3.amazonaws.com/tripdata/2014-citibike-tripdata.zip"
link_2015 = "https://s3.amazonaws.com/tripdata/2015-citibike-tripdata.zip"
link_2016 = "https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip"
link_2017 = "https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip"
link_2018 = "https://s3.amazonaws.com/tripdata/2018-citibike-tripdata.zip"
link_2019 = "https://s3.amazonaws.com/tripdata/2019-citibike-tripdata.zip"
link_2020 = "https://s3.amazonaws.com/tripdata/2020-citibike-tripdata.zip"
link_2021 = "https://s3.amazonaws.com/tripdata/2021-citibike-tripdata.zip"
link_2022 = "https://s3.amazonaws.com/tripdata/2022-citibike-tripdata.zip"
link_2023 = "https://s3.amazonaws.com/tripdata/2023-citibike-tripdata.zip"


file_2014 = "file_2014.zip"
file_2015 = "file_2015.zip"
file_2016 = "file_2016.zip"
file_2017 = "file_2017.zip"
file_2018 = "file_2018.zip"
file_2019 = "file_2019.zip"
file_2020 = "file_2020.zip"
file_2021 = "file_2021.zip"
file_2022 = "file_2022.zip"
file_2023 = "file_2023.zip"

# List for the couples (file, link)
data_list = []

data_list.append((file_2014, link_2014))
data_list.append((file_2015, link_2015))
data_list.append((file_2016, link_2016))
data_list.append((file_2017, link_2017))
data_list.append((file_2018, link_2018))
data_list.append((file_2019, link_2019))
data_list.append((file_2020, link_2020))
data_list.append((file_2021, link_2021))
data_list.append((file_2022, link_2022))
data_list.append((file_2023, link_2023))

years = range(2014, 2024)

for filename,link in data_list:
    if os.path.exists(os.path.join(path_data, filename)):
        print('The file %s already exists.' % os.path.join(path_data, filename))
    else:
        r = requests.get(link)
        with open(os.path.join(path_data, filename), 'wb') as f:
            f.write(r.content)
        print(f'Downloaded file {os.path.join(path_data, filename)}.')
# 11 min

## Unzipping

In [ ]:
import zipfile

#def unzip_file(file_path, dest_folder):
 #   if not os.path.exists(dest_folder):
  #      with zipfile.ZipFile(file_path, 'r') as zip_ref:
   #         zip_ref.extractall(dest_folder)
   #         print(f"Unzipped: {file_path} to {dest_folder}")
    #else:
   #    print(f"Already unzipped: {file_path}")
#for file_path in data_list:
 #   unzip_file( f"{path_data}/{file_path[0]}", unzipped_folder)


## create a folder to store the data
#unzip_folder = "unzip_data"

unzipped_folder = "unzipped_data"
## the path
path_data = os.path.join(os.getcwd(), "data")

## list all the files in the data folder with path
path_list = [os.path.join(path_data, file) for file in os.listdir(path_data)]
#print(path_list)

def unzip_files(paths_list, destination_folder):
    ## check if the folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        ## unzip the files
        zipfile_list = [file for file in path_list if file.endswith(".zip")]
        for file in zipfile_list:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                ## test if the file is not already unzipped else prinr a message
                if os.path.exists(os.path.join(destination_folder, os.path.basename(file).replace(".zip", ""))):
                    print(f"{file} is already unzipped")
                else:
                    zip_ref.extractall(destination_folder)
                    print(f"Extracted {file} to {destination_folder}")
    else:
        print(f"{destination_folder} alreday exists")
        
        
            

unzip_files(path_list, unzipped_folder) 

In [ ]:
# Return the path of the first directory. The first directory look like "2014-citibike-tripdata"
def get_first_directory_path_with_year(year):
    return f"{year}-citibike-tripdata"


In [ ]:
import os

# Get all CSV files in the year directory
def get_all_csv_files(base_dir):
    csv_files = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

## Let's create 1 dataframe per year by merging all the dataframe per year

We add year and month for future partitions

In [ ]:
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.appName("Projet 2 2024").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

years = range(2014, 2024)
unzipped_folder = "unzipped_data"

# We add columns and return df
def read_and_process_file(file_path):
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df = df.withColumn("file_name", sf.input_file_name())
    df = df.withColumn("month", sf.regexp_extract("file_name", r"(\d{4})(\d{2})-", 2))
    df = df.withColumn("year", sf.regexp_extract("file_name", r"(\d{4})(\d{2})-", 1))
    df = df.drop("file_name")
    return df

all_df_by_year = []
# We create the dataframe per year
for year in years:
    input_dir = os.path.join(unzipped_folder, get_first_directory_path_with_year(year))
    
    csv_files = get_all_csv_files(input_dir)
    
    dfs = []
    for file in csv_files:
        df = read_and_process_file(file)
        dfs.append(df)
    
    # Read all CSV files into a single DataFrame
    if dfs:
        year_df = dfs[0]
        for df in dfs[1:]:
            year_df = year_df.unionByName(df, allowMissingColumns=True)
        all_df_by_year.append((year, year_df))

# 6 min

In [ ]:
def get_df_by_year(all_df,year):
    for year_df,df in all_df:
        if year_df == year:
            return df
    return None


## Let's show all the datas per year and see what we have to clean

In [ ]:
for year in years:
    get_df_by_year( all_df_by_year,year).show(1)

## Analysis

We can see that almost all year have different columns name for the same columns and we have some duplicate
-   member_casual is the same as usertype but with different value so we will save member_casual and convert usertype value into member_casual ones
-   we also have started_at and startime and a lot of columns that says the same thing
-   birth year, bike_id and gender disappear after 2021
-   ride_id and rideable_type appear after 2021
So At the end we will have the same columns with all dataframes and the columns are:
rideable_type, started_at,ended_at, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat,end_lng, member_casual, month, year, gender

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.functions import  coalesce,lit
def process_year_2014_2015_2018_2019_2020(df):
    df = (df\
        .withColumnRenamed("starttime", "started_at")\
        .withColumnRenamed("stoptime", "ended_at")\
        .withColumnRenamed("start station id", "start_station_id")\
        .withColumnRenamed("start station name", "start_station_name")\
        .withColumnRenamed("start station latitude", "start_lat")\
        .withColumnRenamed("start station longitude", "start_lng")\
        .withColumnRenamed("end station id", "end_station_id")\
        .withColumnRenamed("end station name", "end_station_name")\
        .withColumnRenamed("end station latitude", "end_lat")\
        .withColumnRenamed("end station longitude", "end_lng")\
        .withColumnRenamed("usertype", "member_casual")\
        .withColumnRenamed("birth year", "birth_year")\
        .drop("bikeid")
        .drop("tripduration")
        .withColumn("rideable_type", lit(None))
    )
    return df

def process_year_2016(df):
    # Fusion of duplicate columns
    df = (df
        .withColumn("tripduration", coalesce(df["Trip Duration"], df["tripduration"]))
        .withColumn("starttime", coalesce(df["Start Time"], df["starttime"]))
        .withColumn("stoptime", coalesce(df["Stop Time"], df["stoptime"]))
        .withColumn("bikeid", coalesce(df["Bike ID"], df["bikeid"]))
        .withColumn("usertype", coalesce(df["User Type"], df["usertype"]))
        .drop("Trip Duration", "Start Time", "Stop Time", "Bike ID", "User Type")
    )
    
    # Changing columns names
    df = (df
        .withColumnRenamed("starttime", "started_at")
        .withColumnRenamed("stoptime", "ended_at")
        .withColumnRenamed("Start Station ID", "start_station_id")
        .withColumnRenamed("Start Station Name", "start_station_name")
        .withColumnRenamed("Start Station Latitude", "start_lat")
        .withColumnRenamed("Start Station Longitude", "start_lng")
        .withColumnRenamed("End Station ID", "end_station_id")
        .withColumnRenamed("End Station Name", "end_station_name")
        .withColumnRenamed("End Station Latitude", "end_lat")
        .withColumnRenamed("End Station Longitude", "end_lng")
        .withColumnRenamed("usertype", "member_casual")
        .withColumnRenamed("Birth Year", "birth_year")\
        .withColumnRenamed("Gender", "gender")\
        .drop("bikeid")
        .drop("tripduration")
        .withColumn("rideable_type", lit(None)) 
    )
    return df

def process_year_2017(df):

    df = (df
        .withColumn("tripduration", coalesce(df["tripduration"], df["Trip Duration"]))
        .withColumn("starttime", coalesce(df["starttime"], df["Start Time"]))
        .withColumn("stoptime", coalesce(df["stoptime"], df["Stop Time"]))
        .withColumn("bikeid", coalesce(df["bikeid"], df["Bike ID"]))
        .withColumn("usertype", coalesce(df["usertype"], df["User Type"]))
        .drop("Trip Duration", "Start Time", "Stop Time", "Bike ID", "User Type")
    )
    
    df = (df
        .withColumnRenamed("starttime", "started_at")
        .withColumnRenamed("stoptime", "ended_at")
        .withColumnRenamed("start station id", "start_station_id")
        .withColumnRenamed("start station name", "start_station_name")
        .withColumnRenamed("start station latitude", "start_lat")
        .withColumnRenamed("start station longitude", "start_lng")
        .withColumnRenamed("end station id", "end_station_id")
        .withColumnRenamed("end station name", "end_station_name")
        .withColumnRenamed("end station latitude", "end_lat")
        .withColumnRenamed("end station longitude", "end_lng")
        .withColumnRenamed("usertype", "member_casual")
        .withColumnRenamed("birth year", "birth_year")
        .drop("bikeid")
        .drop("tripduration")
        .withColumn("rideable_type", lit(None)) 
    )
    
    return df

def process_year_2021(df):

    df = (df
        .withColumn("started_at", coalesce(df["started_at"], df["starttime"]))
        .withColumn("ended_at", coalesce(df["ended_at"], df["stoptime"]))
        .withColumn("start_station_name", coalesce(df["start_station_name"], df["start station name"]))
        .withColumn("start_station_id", coalesce(df["start_station_id"], df["start station id"]))
        .withColumn("end_station_name", coalesce(df["end_station_name"], df["end station name"]))
        .withColumn("end_station_id", coalesce(df["end_station_id"], df["end station id"]))
        .withColumn("member_casual", coalesce(df["member_casual"], df["usertype"]))
        
        .withColumn("start_lat", coalesce(df["start_lat"], df["start station latitude"]))
        .withColumn("start_lng", coalesce(df["start_lng"], df["start station longitude"]))
        .withColumn("end_lat", coalesce(df["end_lat"], df["end station latitude"]))
        .withColumn("end_lng", coalesce(df["end_lng"], df["end station longitude"]))
        .drop("ride_id","tripduration", "starttime", "stoptime", "start station id", "start station name", "start station latitude", "start station longitude", "end station id", "end station name", "end station latitude", "end station longitude", "bikeid", "usertype")
    )
    
    df = df.withColumnRenamed("birth year", "birth_year")
    return df

def process_year_2022_2023(df):
    df = (df
        .withColumn("birth_year", lit(None))
        .withColumn("gender", lit(None))
        .drop("ride_id")
    )
    return df

# Process data per year to have the same columns after
def process_using_year(df, year):
    if year in [2014, 2015, 2018, 2019, 2020]:
        return process_year_2014_2015_2018_2019_2020(df)
    elif year == 2016:
        return process_year_2016(df)
    elif year == 2017:
        return process_year_2017(df)
    elif year == 2021:
        return process_year_2021(df)
    elif year in [2022, 2023]:
        return process_year_2022_2023(df)
    else:
        print("Year {} not supported.".format(year))
        return df  # Retourne le DataFrame non modifié si l'année n'est pas prise en charge

# Adapting usertype values to member_casual values
def process_member_casual(df):
    df = df.withColumn("member_casual", 
        when(df["member_casual"] == "Subscriber", "member")
        .when(df["member_casual"] == "Customer", "casual")
        .otherwise(df["member_casual"])
    )
    return df

input_date_format = 'MM/dd/yyyy HH:mm:ss'

output_date_format = 'yyyy-MM-dd HH:mm:ss'

from pyspark.sql.types import TimestampType
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Changing all dates into timestamp
def process_timestamp_format(df):
    # Getting the types
    started_at_type = df.select("started_at").dtypes[0][1]
    ended_at_type = df.select("ended_at").dtypes[0][1]
    
    if started_at_type != 'timestamp':
        # print(f"Pas timeStamp mais {started_at_type}")
        df = df.withColumn('started_at', to_timestamp(df['started_at'], input_date_format))
    if ended_at_type != 'timestamp':
        df = df.withColumn('ended_at', to_timestamp(df['ended_at'], input_date_format))
    
    return df

column_types = {
    'birth_year': 'string',
    'gender': 'string', 
    'rideable_type': 'string', 
    'started_at': 'timestamp', 
    'ended_at': 'timestamp',
    'start_station_name': 'string', 
    'start_station_id': 'string',
    'end_station_name': 'string', 
    'end_station_id': 'string',
    'start_lat': 'double', 
    'start_lng': 'double', 
    'end_lat': 'double',
    'end_lng': 'double', 
    'member_casual': 'string', 
    'month': 'integer', 
    'year': 'integer'
}

# Applying types
def apply_column_types(df):
    for column, data_type in column_types.items():
        df = df.withColumn(column, df[column].cast(data_type))
    return df
    

In [ ]:

for i, (year, year_df) in enumerate(all_df_by_year):
    print(f"Annee {year}")
    year_df = process_using_year(year_df, year)
    year_df = process_member_casual(year_df)
    year_df = process_timestamp_format(year_df)
    year_df = apply_column_types(year_df)
    
    all_df_by_year[i] = (year,year_df)
    
# sec

In [ ]:
column_sets = [set(df.columns) for _, df in all_df_by_year]

# Just checking if all columns names are good
if all(column_set == column_sets[0] for column_set in column_sets):
    print("Same columns name")
else:
    print("Not Same columns name")

# Checking if all dataframes have the same amount of columns
num_columns = len(all_df_by_year[0][1].columns)
if all(len(df.columns) == num_columns for _, df in all_df_by_year):
    print("Same number of column")
else:
    print("Not Same number of columns")

## Cheking the datas

In [ ]:
for year,year_df in all_df_by_year:
    year_df.select('started_at', 'ended_at').show(1)   

In [ ]:
get_df_by_year( all_df_by_year,2019).count(),get_df_by_year( all_df_by_year,2023).count()

In [ ]:
# Ici c'est juste pour chercher les null d'un df
df = get_df_by_year( all_df_by_year,2019)
df.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
# 30s

In [ ]:
df_2014 = get_df_by_year( all_df_by_year,2014)
df_2023 = get_df_by_year( all_df_by_year,2023)
df_2021 = get_df_by_year( all_df_by_year,2021)
df_2020 = get_df_by_year( all_df_by_year,2020)

f_df14 = df_2014.filter(df_2014["start_station_name"] == "Suffolk St & Stanton St")
f_df23 = df_2023.filter(df_2023["start_station_name"] == "Suffolk St & Stanton St")
f_df21 = df_2021.filter(df_2021["start_station_name"] == "Suffolk St & Stanton St")
f_df20 = df_2020.filter(df_2020["start_station_name"] == "Suffolk St & Stanton St")

# Results
f_df14.show(1),f_df21.show(1),f_df20.show(1),f_df23.show(1)

### Results

We can see statiosn have different ids starting at 2022 and slightly changes on latitude and longitude but we won't touch it

## Now let's union all dataframes to have only one

In [ ]:
def union_all_dfs_by_name(all_df_by_year):
    merged_df = all_df_by_year[0][1]
    for _, df in all_df_by_year[1:]:
        merged_df = merged_df.unionByName(df)
    return merged_df

merged_df = union_all_dfs_by_name(all_df_by_year)

In [ ]:
merged_df.show(2),merged_df.tail(2)

## Saving to parquet using column year and month

In [ ]:
## partioning our data with a range partition of years and months


## lets have a vieuw of the data mergeddf   
merged_df.printSchema()
merged_df.show(2)


In [ ]:
## patrtioning the data by year and month
#years = range(2014, 2024)

#months = range(1, 13)

# Partitioning the data by year 
#merged_df = merged_df.repartition("years")

In [ ]:

## the actual number of partitions for the merged_df
num_partitions = merged_df.rdd.getNumPartitions()
print("Number of partitions: ", num_partitions)

In [ ]:

## lets have the numer of rows and columns in the merged_df
num_rows = merged_df.count()
num_cols = len(merged_df.columns)
print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)


In [ ]:
parquet_path = "./trip_data.parquet"
# parquet_path = "./parquet_data"
partition_columns = ["year", "month"]
if not os.path.exists(parquet_path):
    merged_df.write.partitionBy(*partition_columns).parquet(parquet_path)
    print(f"Data written to {parquet_path}")
else:
    print(f"Path {parquet_path} already exists. No data written.")
    
#merged_df.write.partitionBy(*partition_columns)

# 40 min

In [ ]:
# merged_df.unpersist()
# for year,df in all_df_by_year:
#     df.unpersist()

In [ ]:
# from IPython.display import display, Javascript

# def restart_kernel():
#     display(Javascript('IPython.notebook.kernel.restart()'))

# restart_kernel()

# Storage level Questions

## 1. Storage level

In [ ]:
print(merged_df.storageLevel)

## 2. Number of Partitions

In [ ]:
print(merged_df.rdd.getNumPartitions())

## 3. We can easily tune the number of partiton with 

``` spark.read.option("numPartitions", the_number_we_want).csv("path/to/csv") ```

##  4. Why modify the number

Usually we modify this number to optimize the parallelism of the jobs, ensuring that all executors are effectively utilized according o the size of the data. Ex: With small datas, its a bad idea to have a lot of partitions. But with large dataset it performs very good with lot of partitions.

# Organizing into star Schema

## Let's restart spark first
We had a lot of memory problems so we just restart spark

In [ ]:
spark.stop()

In [ ]:
import os
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .appName("Projet 2 2024") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.shuffle.partitions", 100) \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()
# .config("spark.memory.fraction", "0.8")\
# .config("spark.memory.storageFraction", "0.3")\

In [ ]:
parquet_path = "./trip_data.parquet"
parquet_df = spark.read.parquet(parquet_path)
parquet_df.show(2)

In [ ]:
parquet_df.count()

## Stations

In [ ]:
from pyspark.sql.functions import *

start_stations = parquet_df.select(col("start_station_id").alias("station_id"),
    col("start_station_name").alias("station_name")).dropDuplicates(["station_id"])
end_stations = parquet_df.select(col("end_station_id").alias("station_id"),
    col("end_station_name").alias("station_name")).dropDuplicates(["station_id"])

station_id_name = start_stations.union(end_stations).dropDuplicates(["station_id"])

start_coordinates = parquet_df.select(col("start_station_id").alias("station_id"),
    col("start_lat").alias("lat"),
    col("start_lng").alias("lng")).dropDuplicates(["station_id"])
end_coordinates = parquet_df.select(col("end_station_id").alias("station_id"),
    col("end_lat").alias("lat"),
    col("end_lng").alias("lng")).dropDuplicates(["station_id"])

station_coordinates = start_coordinates.dropDuplicates(["station_id"])

station_df = station_id_name.join(station_coordinates, on="station_id", how="inner") \
    .select("station_id", "station_name", "lat", "lng")
# start_stations.unpersist()
# end_stations.unpersist()
# start_coordinates.unpersist()
# end_coordinates.unpersist()
# import gc

# del start_stations
# del end_stations
# del start_coordinates
# del end_coordinates

# # Force garbage collector à free memory
# gc.collect()

## Trips

In [ ]:
# Extract trip information for the event table
trips_df = parquet_df.select(
    "started_at",
    "ended_at",
    "start_station_id",
    "end_station_id",
    "rideable_type",
    "member_casual",
    "year",
    "month",
    "gender"
)


In [ ]:
station_df.persist()
#2 min 40

In [ ]:
station_df.count()

In [ ]:
trips_df.show(5)

In [ ]:
station_df.show(5)

# Computing and showing plot

We limit the dataset  because of space problems and it take too many times to compute (20 min)

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
from geopy.distance import geodesic
import plotly.express as px

# plot_df = parquet_df.dropna(subset=["started_at"])
# plot_df = plot_df.dropna(subset=["ended_at"])

def calculate_distance(start_lat, start_lng, end_lat, end_lng):
    start_coords = (start_lat, start_lng)
    end_coords = (end_lat, end_lng)
    return geodesic(start_coords, end_coords).kilometers

# Just une lambda and we force type to be Double
distance_udf = udf(calculate_distance, DoubleType())

def trip_distance_each_day_of_week(year_df):
    
    plot_df = year_df.withColumn("trip_distance_km", distance_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng")))

    plot_df = plot_df.withColumn("day_of_week", date_format(col("started_at"), "EEEE"))
    
    plot_df = plot_df.select("trip_distance_km", "day_of_week")
    
    plot_df_pandas = plot_df.limit(150000).toPandas()  # Limiting for performance

    fig = px.histogram(plot_df_pandas, x="day_of_week", y="trip_distance_km", 
        title='Trip Distance by Day of Week',
        labels={'trip_distance_km': 'Trip Distance (km)', 'day_of_week': 'Day of the Week'})

    fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']})
    fig.show()

## Trip distribution each day of week

### Trip distribution pre-covid

In [ ]:
df_2018 = parquet_df.filter(parquet_df['year'] == 2018)
trip_distance_each_day_of_week(df_2018)

Of course we limited the dataset so we are missing some days

### Trip distribution post-covid

In [ ]:
df_2022 = parquet_df.filter(parquet_df['year'] == 2022)
trip_distance_each_day_of_week(df_2022)


The dataset is better distributed past-covid, we can see modr distance the closer we are to the weekend and it decrease as the weekend is ending. The post-covid ones does not telle us much be cause we dont have all the days

In [ ]:
# Add count columns
def trip_per_couple_location(df_year):
    plot_df = df_year.groupBy("start_station_name", "end_station_name").count()
    plot_df.limit(150000)
    plot_df = plot_df.toPandas()
    pd = plot_df
    import plotly.express as px

    import plotly.graph_objects as go
    plot_df = plot_df.pivot(index='start_station_name', columns='end_station_name', values='count')

    fig = go.Figure(data=go.Heatmap(
        z=plot_df.values,
        x=plot_df.columns,
        y=plot_df.index,
        colorscale='Viridis'))

    fig.update_layout(
        title='Trip Counts Heatmap for Start and End Station Pairs',
        xaxis=dict(title='End Station'),
        yaxis=dict(title='Start Station'),
        autosize=False,
        width=800,
        height=800
    )

    fig.show()


## Trips per pickup/dropoff location

### Trips per pickup/dropoff location prec - covid (2018)

In [ ]:
trip_per_couple_location(df_2018)

### Trips per pickup/dropoff location post - covid (2022)

In [ ]:
trip_per_couple_location(df_2022)

Hard to read but it's the best of all we tried

In [ ]:
def map_gender(gender):
    if gender == '1':
        return 'Male'
    elif gender == '2':
        return 'Female'
    else:
        return 'Unknown'

map_gender_udf = udf(map_gender, StringType())

def trip_distance_distribution_by_gender(df):
    df = df.withColumn('gender_label', map_gender_udf(col('gender')))

    df = df.withColumn("trip_distance_km", distance_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng")))
    df = df.groupBy('gender_label').agg({'trip_distance_km': 'sum'})
    df_pd = df.toPandas()

    fig = px.bar(df_pd, x='gender_label', y='sum(trip_distance_km)', title='Total Trip Distance by Gender')
    fig.update_layout(xaxis_title='Gender', yaxis_title='Total Trip Distance (km)')
    fig.show()


## Trip distance distribution per gender

In [ ]:
df_2020 = parquet_df.filter(parquet_df['year'] == 2020)
plot_df = df_2020.limit(70000)
trip_distance_distribution_by_gender(plot_df)

## Trip distribution for age ranges

In [ ]:
from pyspark.sql.functions import radians, sin, cos, sqrt, asin, col

def calculate_distance2(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Formula to calculate distance
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    distance = 2 * asin(sqrt(a)) * 6371  # Radius of earth in kilometers

    return distance


In [ ]:
import datetime
from pyspark.sql.functions import year, current_date
# def calculate_age(birth_year):
#     current_year = datetime.datetime.now().year
#     return current_year - int(birth_year)

def assign_age_range(age):
    if age >= 15 and age <= 24:
        return '15-24'
    elif age >= 25 and age <= 44:
        return '25-44'
    elif age >= 45 and age <= 54:
        return '45-54'
    elif age >= 55 and age <= 64:
        return '55-64'
    else:
        return '65+'


#df_2020_age = df_2020.withColumn('age', year(current_date()) - df_2020['birth_year'])
#assign_age_range_udf = udf(assign_age_range, StringType())
#df_with_age_range = df_2020_age.withColumn('age_range', assign_age_range_udf(col('age')))
#df_with_trip_distance = df_with_age_range.withColumn('trip_distance_km', calculate_distance2(col('start_lat'), col('start_lng'), col('end_lat'), col('end_lng')))
#trip_distance_distribution = df_with_trip_distance.groupBy('age_range').agg({'trip_distance_km': 'collect_list'})

def trip_distance_distribution_by_age(df):
    df_with_age = df.withColumn('age', year(current_date()) - df_2020['birth_year'])

    assign_age_range_udf = udf(assign_age_range, StringType())
    df_with_age_range = df_with_age.withColumn('age_range', assign_age_range_udf(col('age')))

    # df_with_trip_distance = df_with_age_range.withColumn('trip_distance_km', calculate_distance2(df_with_age_range['start_lat'], df_with_age_range['start_lng'], df_with_age_range['end_lat'], df_with_age_range['end_lng']))
    df_with_trip_distance = df_with_age_range.withColumn('trip_distance_km', distance_udf(col('start_lat'), col('start_lng'), col('end_lat'), col('end_lng')))
    trip_distance_distribution = df_with_trip_distance.groupBy('age_range').agg({'trip_distance_km': 'collect_list'})
    return trip_distance_distribution



In [ ]:
import plotly.express as px

def visualize_trip_distance_distribution(trip_distance_distribution):
    trip_distance_distribution_pd = trip_distance_distribution.toPandas()

    fig = px.box(trip_distance_distribution_pd, x='age_range', y='collect_list(trip_distance_km)',
        title='Trip Distance Distribution by Age Range',
        labels={'age_range': 'Age Range', 'collect_list(trip_distance_km)': 'Trip Distance (km)'})

    fig.update_layout(xaxis_title='Age Range', yaxis_title='Trip Distance (km)')

    # Show the plot
    fig.show()
    

In [ ]:
tri= trip_distance_distribution_by_age(df_2020.limit(1000))
tri.show(5)

In [ ]:
# trip_distance_distribution = trip_distance_distribution_by_age(df_2020.limit(1000))

# visualize_trip_distance_distribution(trip_distance_distribution)



In [ ]:
trip_distance_distribution = trip_distance_distribution_by_age(df_2020.limit(150000))
def visualize_trip_distance_distribution_histogram(trip_distance_distribution):
    trip_distance_distribution_pd = trip_distance_distribution.toPandas()

    trip_distance_distribution_pd = trip_distance_distribution_pd.explode('collect_list(trip_distance_km)')

    fig = px.histogram(trip_distance_distribution_pd, x='collect_list(trip_distance_km)', color='age_range',
        title='Trip distribution by age range',
        labels={'collect_list(trip_distance_km)': 'Distance du Trajet (km)', 'age_range': 'Age range'},
        nbins=50, barmode='overlay')

    fig.update_layout(xaxis_title='Distance du Trajet (km)', yaxis_title='Fréquence')

    fig.show()

# Exemple d'utilisation :
visualize_trip_distance_distribution_histogram(trip_distance_distribution)

Contrary to what one might think 25-44 age range uses the bikes more than 15-24

In [ ]:
def trip_distance_distribution_by_rideable_type(df):
    df = df.filter(df['rideable_type'].isNotNull())

    df = df.withColumn("trip_distance_km", distance_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng")))

    df = df.groupBy('rideable_type').agg({'trip_distance_km': 'sum'})
    df.limit(100)
    fig = px.bar(df.toPandas(), x='rideable_type', y='sum(trip_distance_km)', title='Total Trip Distance by bike type')
    fig.update_layout(xaxis_title='bike type', yaxis_title='Total Trip Distance (km)')
    fig.show()


## Post covid trip distribution for bikes types(non null): 2021  2023

In [ ]:
df_2023= parquet_df.filter(parquet_df['year'] == 2023)
df_trip_bike_2023 = trip_distance_distribution_by_rideable_type(df_2023.limit(15000))

after covid we can see that the distribution is unbanlanced, we have a lot of classic bikes

## During covid trip distribution for bikes types (non null): 2021 

In [ ]:
df_2021= parquet_df.filter(parquet_df['year'] == 2021)
df_trip_bike_2021 = trip_distance_distribution_by_rideable_type(df_2021.limit(15000))

Trip distance was very high during covid compared to 2023 (actually weird). Possible raisons are the transport were shut down during covid

# Assessing seasonalities and looking at time series

##  The number of pickups/docks

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dayofweek, hour, count, date_format
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = parquet_df.withColumn('day_of_week', date_format(col('started_at'), 'E'))
df = df.withColumn('hour_of_day', hour(col('started_at')))

grouped_df = df.groupBy('day_of_week', 'hour_of_day').agg(count('*').alias('pickups_docks'))

pandas_df = grouped_df.toPandas()

ordered_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
pandas_df['day_of_week'] = pd.Categorical(pandas_df['day_of_week'], categories=ordered_days, ordered=True)

pivot_df = pandas_df.pivot_table(index='hour_of_day', columns='day_of_week', values='pickups_docks', fill_value=0)
pivot_df = pivot_df.astype(int)
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_df, cmap='viridis', annot=True, fmt='d')
plt.title('Number of Pickups/Docks by Day of the Week and Hour of the Day')
plt.xlabel('Day of the Week')
plt.ylabel('Hour of the Day')
plt.show()
#   1 - 6min

As we could have guessed, we have more trips during workdays and hours like 8am (when people goes to work) and 4-6 pm (when people gets off of work). Notice that here we used the dataframe of all the years because it does not long long time compared to the others.

## The average distance

In [ ]:
df = df_2022.limit(550000)
df = df.withColumn("distance_km", distance_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng")))
df = df.withColumn('day_of_week', date_format(col('started_at'), 'E'))
df = df.withColumn('hour_of_day', hour(col('started_at'))) 

df_avg_distance = df.groupBy('day_of_week', 'hour_of_day').agg(mean('distance_km').alias('avg_distance_km'))

pandas_avg_distance = df_avg_distance.toPandas()

pandas_avg_distance['day_of_week'] = pd.Categorical(pandas_avg_distance['day_of_week'], categories=ordered_days, ordered=True)

pivot_avg_distance = pandas_avg_distance.pivot_table(index='hour_of_day', columns='day_of_week', values='avg_distance_km', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_avg_distance, cmap='viridis', annot=True, fmt='.2f')
plt.title('Average Distance by Day of the Week and Hour of the Day')
plt.xlabel('Day of the Week')
plt.ylabel('Hour of the Day')
plt.show()
# 2 min - 10 min

Seems pretty normal beside the weird peak on tuesday and saturday morning at 3 am. Of course we lilmited the data be cause of long loading time crashing the kernel so it's possible that those peaks are biais

## The average trip duration

In [ ]:
df = df_2022.limit(550000)
df = df.withColumn("distance_km", distance_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng")))

df = df.withColumn('trip_duration', unix_timestamp('ended_at') - unix_timestamp('started_at'))
df = df.withColumn('trip_duration', col('trip_duration') / 60)
df = df.withColumn('day_of_week', date_format(col('started_at'), 'E')) 
df = df.withColumn('hour_of_day', hour(col('started_at'))) 

df_avg_duration = df.groupBy('day_of_week', 'hour_of_day').agg(mean('trip_duration').alias('avg_trip_duration'))

pandas_avg_duration = df_avg_duration.toPandas()

pandas_avg_duration['day_of_week'] = pd.Categorical(pandas_avg_duration['day_of_week'], categories=ordered_days, ordered=True)

pivot_avg_duration = pandas_avg_duration.pivot_table(index='hour_of_day', columns='day_of_week', values='avg_trip_duration', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_avg_duration, cmap='viridis', annot=True, fmt='.2f')
plt.title('Average Trip Duration by Day of the Week and Hour of the Day in minutes')
plt.xlabel('Day of the Week')
plt.ylabel('Hour of the Day')
plt.show()

We have long trips duration on weekend probably because people who use bikes on work days to go to work don't take long trips. Either the work office is close so they use the bike for smal trip duration or the use the bike just to get to the station and take transport so small trip duration too.

The average number of ongoing trips

In [ ]:
from pyspark.sql.functions import col, hour, date_format, count, mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.window import Window

df = df_2022.limit(5000000)
df = df.withColumn('day_of_week', date_format(col('started_at'), 'E'))
df = df.withColumn('hour_of_day', hour(col('started_at')))
df = df.withColumn('date', to_date(col('started_at')))

# Partition so we add to each row a colums that give number of trips for that day
windowSpec = Window.partitionBy('date', 'hour_of_day')
df = df.withColumn('num_trips', count('started_at').over(windowSpec))
df = df.groupBy('hour_of_day', 'day_of_week').agg(mean('num_trips').alias('avg_trips'))

pandas_avg_trips = df.toPandas()

pandas_avg_trips['day_of_week'] = pd.Categorical(pandas_avg_trips['day_of_week'], categories=ordered_days, ordered=True)

pivot_avg_departures = pandas_avg_trips.pivot_table(index='hour_of_day', columns='day_of_week', values='avg_trips', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_avg_departures, cmap='viridis', annot=True, fmt='.2f')
plt.title('Average Number of trips by Day of the Week and Hour of the Day')
plt.xlabel('Day of the Week')
plt.ylabel('Hour of the Day')
plt.show()


It cannot get more clear than that, rush hours of work days have a lot of trips. So does the weekend !!!

# Monitor job execution

##  1. Explain

In [ ]:
df.explain(True)

##  2. Analysing Analyzed and Optimized plan


### Differences between analyzed and optimized logical plan

-   The Optimized Logical Plan is  samller
-   The Optimized Logical Plan  applies the limit we did for the dataframes earlier compared to Analyzed Logical Plan
-   The Optimized Logical Plan retained only necessary columns
-   and more ...

###  RDBMS
Spark would perform a little bit better than RDBMS

##  3. Analysing physical and Optimized plan
The physical logical plan specifies the actual methods that will be used to process the data such as join shuffles etc ...  It also uses not RDBMS keywords like AdaptiveSparkPlan (An optimized plan use by spark based on runtime statistics), Project(Selection of columns from dataframe ), Sort (is order by in RDBMS),Exchange, ColumnarToRow etc ...

##  4. Inspecting spark UI

In [ ]:
df = df_2022.limit(5000000)
df = df.withColumn('day_of_week', date_format(col('started_at'), 'E'))
df = df.withColumn('hour_of_day', hour(col('started_at')))
df = df.withColumn('date', to_date(col('started_at')))

windowSpec = Window.partitionBy('date', 'hour_of_day')
df = df.withColumn('num_trips', count('started_at').over(windowSpec))
df = df.groupBy('hour_of_day', 'day_of_week').agg(mean('num_trips').alias('avg_trips'))
df.show(10)

Inspecting the UI, we had 2 stages. Hash agrregates is used to group large datasets by value. Hash partitioning shufffles and repartition the data whil ensuring data with same keys ends up on the same node

## 5. Shuffle operations

We have 2 shuffle  (Exchange) int the physical plan

##  6. Stages and tasks

Stages are created based on shuffle ans whithin each stage we have tasks that represents the parallel execution unitsof a partition of the data. Our first stage has 16 tasks and the second one have 1 task.

# Diving into spatial information problems

## 1. Map of trips betweeen stations

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import *
from geopy.distance import geodesic
# import plotly.express as px
df_2019 = parquet_df.filter(parquet_df['year'] == 2019)
df_filtered = df_2022.limit(7000000)
# Agrégation per stations couple
station_pairs = df_filtered.groupBy("start_station_name", "start_lat", "start_lng", "end_station_name", "end_lat", "end_lng") \
    .agg(count("*").alias("trip_count"))
station_pairs_pd = station_pairs.toPandas()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import contextily as ctx
import numpy as np

# Geopanda need gemetry column so we add 
station_pairs_pd['geometry'] = station_pairs_pd.apply(
    lambda row: LineString([(row['start_lng'], row['start_lat']), (row['end_lng'], row['end_lat'])]), axis=1)

gdf = gpd.GeoDataFrame(station_pairs_pd, geometry='geometry')
# CRS)
gdf.crs = "EPSG:4326"
gdf = gdf.to_crs(epsg=3857)


In [ ]:

# plot map
fig, ax = plt.subplots(figsize=(12, 8))

# plot lines
gdf.plot(ax=ax, column='trip_count', cmap='viridis', linewidth=1, legend=True)

# base map
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, reset_extent=False)
ax.autoscale()

plt.title('Heatmap of trips at New York')
plt.xlabel('Longitude')
plt.ylabel('Latitude')


plt.show()

The map does not show that much of yellow color but we can clearly see from the select under that we have some couple of stations with trips over 1500. They are just stuck behind the violet ones. (DATA from 2022)

In [ ]:
gdf_filtered = gdf[gdf['trip_count'] > 1500]
gdf_filtered.head(20)

## 2.Interactive Map of trips betweeen stations

In [ ]:
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
# Convert the PySpark DataFrame to a Pandas DataFrame
station_pandas_df = station_df.toPandas()

In [ ]:
# Create a GeoDataFrame from the Pandas DataFrame
# DataFrame with 'lng' and 'lat' columns
geometry = [Point(xy) for xy in zip(station_pandas_df.lng, station_pandas_df.lat)]
geo_station_df = GeoDataFrame(station_pandas_df, geometry=geometry)

In [ ]:
geo_station_df.to_file("output.geojson", driver='GeoJSON')

In [ ]:
# df_2019 = parquet_df.filter(col("year") == 2019).limit(5000000)
# trip_counts = df_2019.groupBy("start_station_id", "end_station_id") \
#     .count() \
#     .withColumnRenamed("count", "trip_count")

In [ ]:
# trip_counts_pd = trip_counts.toPandas()

In [ ]:
import plotly.express as px

# Load the station data
geo_station_df = gpd.read_file("output.geojson")

# # Merge trip counts with station data
# trip_counts_merged = trip_counts_pd.merge(geo_station_df, left_on="start_station_id", right_on="station_id")
# trip_counts_merged = trip_counts_merged.merge(geo_station_df, left_on="end_station_id", right_on="station_id", suffixes=('_start', '_end'))

# # Create the heatmap indexed by start and end stations where the color represents the number of trips count
# fig = px.density_mapbox(trip_counts_merged, lat='lat_start', lon='lng_start', z='trip_count', radius=10,
#     center=dict(lat=40.7128, lon=-74.0060), zoom=10,
#     mapbox_style="open-street-map")
# fig.show()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, hour as extract_hour

## hours from timestamp
df_2019 = df_2019.withColumn("hour", extract_hour(col("started_at")))

## groupz by start and end stations and hour to get trip counts
hourly_trip_counts = df_2019.groupBy("start_station_id", "end_station_id", "hour") \
    .count() \
    .withColumnRenamed("count", "trip_count")

In [ ]:
import geopandas as gpd

# load in geojson file station
geo_station_df = gpd.read_file("output.geojson")

geo_df = gpd.read_file("output.geojson")

# Convert stations data to DataFrame and rename columns for merging
stations_dfs = geo_df[['station_id', 'geometry']]
stations_dfs['latitude'] = geo_df.geometry.y
stations_dfs['longitude'] = geo_df.geometry.x

In [ ]:
# Convert to Pandas DataFrame
stations_pds = stations_dfs[['station_id', 'latitude', 'longitude']]

In [ ]:
# convert to pandas dataframe
hourly_trip_counts_pd = hourly_trip_counts.toPandas()

In [ ]:
# Merge trip counts with start and end station coordinates
hourly_trip_counts_merged = hourly_trip_counts_pd.merge(stations_pds, left_on="start_station_id", right_on="station_id")
hourly_trip_counts_merged = hourly_trip_counts_merged.merge(stations_pds, left_on="end_station_id", right_on="station_id", suffixes=('_start', '_end'))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Create a single DataFrame to store all the traces
all_traces = []

for hr in range(24):
    hourly_data = hourly_trip_counts_merged[hourly_trip_counts_merged['hour'] == hr]
    trace = go.Densitymapbox(
        lat=hourly_data['latitude_start'],
        lon=hourly_data['longitude_start'],
        z=hourly_data['trip_count'],
        radius=10,
        name=f'Hour {hr}',
        visible=(hr == 0)  # Only the first hour is visible initially
    )
    all_traces.append(trace)

# Create the figure
fig = go.Figure(data=all_traces)

# Update layout for mapbox
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=40.7128, lon=-74.0060),
        zoom=10
    ),
    updatemenus=[{
        "buttons": [
            {
                "args": [{"visible": [hr == i for i in range(24)]}],
                "label": f"Hour {hr}",
                "method": "update"
            } for hr in range(24)
        ],
        "direction": "down",
        "showactive": True,
    }],
    sliders=[{
        "steps": [
            {
                "args": [
                    [hr],
                    {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}
                ],
                "label": str(hr),
                "method": "animate"
            } for hr in range(24)
        ],
        "transition": {"duration": 0},
        "x": 0.1,
        "len": 0.9
    }]
)

fig.show()

In [ ]:
# station_df.unpersist()

In [ ]:
# Stop the Spark session
# spark.stop()